###Ingest LapTimes data from bronze to silver

1. Imports

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp

2. Create schema

In [0]:
laptimes_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), False),
        StructField("lap", IntegerType(), False),
        StructField("position", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("milliseconds", IntegerType(), True)
    ]
)

3. Read lap_times/*.csv file

In [0]:
laptimes_raw_df = spark.read \
    .schema(laptimes_schema) \
    .csv("abfss://bronze@formula1faijdl.dfs.core.windows.net/lap_times/*.csv")

4. Transform the circuit dataframe

In [0]:
laptimes_df = laptimes_raw_df \
        .select(
                col("raceId").alias("race_id"),
                col("driverId").alias("driver_id"),
                col("lap"),
                col("position"),
                col("time"),
                col("milliseconds"),
                current_timestamp().alias('ingestion_date')
        )

5. Write the dataframe to silver layer

In [0]:
laptimes_df.write.mode('overwrite').parquet("abfss://silver@formula1faijdl.dfs.core.windows.net/laptimes")